In [ ]:
import qutip as qt

import numpy as np
from matplotlib import pyplot as plt

import jax
import jax.numpy as jnp
from diffrax import Dopri5, PIDController, Dopri8

from qutip_qoc import optimize_pulses, TimeInterval, Objective

# Quantum Optimal Control Task

We want to transform an initial system $X$ through application of $H$ into a desired target $X_\mathrm{trgt}$.

\begin{gather*}
    X(t=0) \underset{H}{\longrightarrow} X_\mathrm{trgt}=X(t=T)
\end{gather*}

The transformation must obay the Schrödinger equation

\begin{gather}
    \partial_{t} X(t) = - \mathrm i \underbrace{\left( A + \sum_{k=0}^{C-1} c_k(t, \vec{\alpha}_k) B_k \right)}_{H} X(t)
\end{gather}

for closed systems. Different settings are possible

\begin{array}{c|c|c|c}
 & \text{drift } A        & \text{controls } B_k & X(t, \alpha) \\\hline
 \text{state transfer}  &  H_d         &  B_k& |\Psi(t, \alpha)>\\
 \text{gate synthesis}       &  H_d         &  B_k & U(t, \alpha)   \\\hline
 \text{state transfer}      &  \hat H_d    &  \hat B_k & \rho(t, \alpha) \\
 \text{gate synthesis}     &  \hat H_d    &  \hat B_k & \hat{U}(t, \alpha)
\end{array}

where super-operators are labeled by a hat. Two possible figures of merit are 
$f_\mathrm{SU} = \mathrm{Re}(g) \text{ and } f_\mathrm{PSU} = |g|$

\begin{gather*}
    g = \frac{1}{\|{X_\mathrm{trgt}} \|} \mathrm{tr} \left[X_\mathrm{trgt}^{\dagger} X(T, \alpha) \right]
\end{gather*}

Open systems must obay the Master equation with Lindbladian $\hat L$ and trace difference $f_\mathrm{TRCDFF} = \mathrm{Re}(d)$ as possible distance measure.

\begin{array}{c|c|c|c}
  & \text{drift } A   & \text{controls } B_k & X(t, \alpha) \\\hline
 \text{state transfer}     &  \hat H_d +
 \mathrm i \hat{L}    &  \hat B_k& \rho(t, \alpha) \\
 \text{map synthesis}         &   \hat H_d + \mathrm i \hat L    &  \hat B_k& \hat{U}(t, \alpha)\\
\end{array}

\begin{gather*}
    d = \frac{1}{2\|{X_\mathrm{trgt}} \|} \mathrm{tr} \left[ (X(T, \alpha) - X_\mathrm{trgt})^\dagger  (X(T, \alpha) - X_\mathrm{trgt})\right]
\end{gather*}

---
# GRadient Ascent Pulse Engineering (GRAPE) [1]
---

Slice time interval $T$ in $M$ chunks of size $\Delta t$, with piecewise constant functions $c_k(t_i, \alpha_{i,k}) = \alpha_{i,k}$ $\forall i \in [1, M]$

\begin{gather*}
H(t_i, \alpha) \approx A + \sum_{k=0}^{C-1} \alpha_{i,k} B_k \\

X(t_i, \alpha) = \underbrace{X_i}_{\exp(-\mathrm i H(t_i, \alpha) \Delta t)}
X_{i-1} \dots X_1 X_0
\end{gather*}

and update parameters according to $\frac{\partial f^{(r)}(X(t, \alpha))}{\partial \alpha_{i,k}}$.

<img src="doc_images\DYNAMO.png" alt="dynamo" width="600"/>

[1] S. Machnes, U. Sander, S. J. Glaser, P. De Fouquieres, A. Gruslys, S. Schirmer, and T. Schulte-Herbrueggen, Comparing, Optimising and Benchmarking Quantum Control Algorithms in a Unifying Programming Framework, Phys. Rev. A. 84, 022305 (2010). arXiv:1011.4874

---
# Chopped RAndom Basis (CRAB)[2]
---

Update initial guess controls $c^{\text{ init}}_k(t, \vec \alpha_k)$ through parameterized update function $g_k(t, \vec \beta_k)$ expanded in some function basis i.e. Fourier:

\begin{align*}
    c^{\text{ update}}_k(t, \vec \alpha_k) &= c^{\text{ init}}_k(t, \vec \alpha_k) \cdot g_k(t, \vec \beta_k) \\ \\
    &= c^{\text{ init}}_k(t, \vec \alpha_k) \left( 1 +
    \sum_{l=1}^{\infty} A_l \sin ( \omega_l t) + B_l \cos ( \omega_l t) \right)
\end{align*}

fix + randomly choose $\omega_l$ and chop to finite basis $l\in[1, L]$

<img src="doc_images\CRAB.png" alt="crab" width="800"/>

[2] Caneva, T. Calarco, & S. Montangero, Chopped random-basis quantum optimization, Phys. Rev. A 84, 022326 (2011). doi:10.1103/PhysRevA.84.022326

---
# Gradient Optimization of Analytic conTrols (GOAT) [3]
---

Taking the derivative of the Schrödinger operator equation w.r.t. the control parameters $\alpha$

\begin{align}
    \mathrm i\partial_{t} X = & \phantom{xx} H X\\
    \mathrm i\partial_{\alpha}\partial_{t} X = &\partial_{\alpha}  H X
\end{align}

results in a coupled system of PDEs. Unrolling the right hand side for implementation

\begin{gather*}
   {\overset{\text{RHS}}{\longrightarrow}} \begin{pmatrix}
        H & 0 \\
        \partial_{{\alpha}}{H}& H \\
    \end{pmatrix}
    \begin{pmatrix}
        X \\
        \partial_{{\alpha}}{X} \\
    \end{pmatrix}
    = \begin{pmatrix}
        H X \\
        (\partial_{{\alpha}}{H}) X + H (\partial_{{\alpha}}{X}) \\
    \end{pmatrix}
    \\ \\ =
    \begin{pmatrix}
        H & 0 & 0 & 0\\
        \partial_{\vec{\alpha}_0}{H}& H & 0 & 0 \\
        \partial_{\vec{\alpha}_1}{H}& 0 & H & 0 \\
        \vdots & 0 & 0 & \ddots \\
    \end{pmatrix}
     \begin{pmatrix}
        X \\
        \partial_{\vec{\alpha}_0}{X} \\
        \partial_{\vec{\alpha}_1}{X} \\
        \vdots \\
    \end{pmatrix}
    \\ \\ 
\end{gather*}




\begin{gather*}
   =
    \begin{pmatrix}
        A + \sum_{k=0}^{C-1} c_k(\vec{\alpha}_k, t) B_k\,& 0 & 0 & 0\\
        \sum_{k=0}^{C-1} B_k\,\partial_{\vec{\alpha}_0} c_k(\vec{\alpha}_k, t)& A + \sum_{k=0}^{C-1} c_k(\vec{\alpha}_k, t) B_k\,& 0 & 0 \\
        \sum_{k=0}^{C-1} B_k\,\partial_{\vec{\alpha}_1} c_k(\vec{\alpha}_k, t)& 0 & A + \sum_{k=0}^{C-1} c_k(\vec{\alpha}_k, t) B_k\,& 0 \\
        \vdots & 0 & 0 & \ddots \\
    \end{pmatrix}
    \\ \\ =
    \begin{pmatrix}
        A & 0 & 0 & 0\\
        0& A & 0 & 0 \\
        0& 0 & A & 0 \\
        0 & 0 & 0 & \ddots \\
    \end{pmatrix}
    +\sum_{k=0}^{C-1}
    \begin{pmatrix}
        c_k(\vec{\alpha}_k, t) B_k\,& 0 & 0 & 0\\
        B_k\,\partial_{\vec{\alpha}_0} c_k(\vec{\alpha}_k, t)&  c_k(\vec{\alpha}_k, t) B_k\,& 0 & 0 \\
        B_k\,\partial_{\vec{\alpha}_1} c_k(\vec{\alpha}_k, t)& 0 & c_k(\vec{\alpha}_k, t) B_k\,& 0 \\
        \vdots & 0 & 0 & \ddots \\
    \end{pmatrix}
    
    
\end{gather*}

\begin{gather*} =
    (\mathbf{I} \otimes A) + \sum_{k=0}^{C-1}
    (\mathbf{I} \otimes B_k) c_k(\vec{\alpha}_k, t) +
    \begin{pmatrix}
        0 & 0 &  \\
        B_0\, \partial_{\vec{\alpha}_0} c_0(\vec{\alpha}_0, t)& 0 & \\
        B_1\,\partial_{\vec{\alpha}_1} c_1(\vec{\alpha}_1, t)& 0  & \\
        \vdots &  & \ddots \\
    \end{pmatrix}
    \\ \\ =
    (\mathbf{I} \otimes A) + \sum_{k=0}^{C-1}
    (\mathbf{I} \otimes B_k) c_k(\vec{\alpha}_k, t) +
    \sum_{k=0}^{C-1} (\mathbf{1}_{0,k+1} \otimes B_k) \partial_{{\alpha}_{k}} c_k(\vec{\alpha}_k, t)
\end{gather*}

where $\mathbf{1}_{0,k+1}$ is the zero square matrix with only one entry in the first column at row index $1 + (k \cdot M + l)$ set to one, and ($l$ index of) $M$ is the number of varaible parameters for each control amplitude $c_k(\vec{\alpha}_k, t)$, i.e. for a superposition of Gaussian pulses with three variable parameters each $\rightarrow M = 3 \cdot m$:
\begin{gather*}
    c_k(\vec{\alpha}_k, t) = \sum_{l=0}^{M-1} A_{k,l} \exp\left({\frac{-(t-\tau_{k,l})^2}{\sigma_{k,l}^2}}\right)
\end{gather*}

Expressing the Hamiltonian in this form makes it comfortable to implement it using QuTiP
\begin{gather*}
    \underbrace{(\mathbf{I} \otimes A)}_{\mathrm{QobjEvo(}\tilde{H}_d,f(x)=1)}
     + \sum_{k=0}^{C-1}\underbrace{(\mathbf{I} \otimes B_k) c_k(\vec{\alpha}_k, t)}_{\mathrm{QobjEvo(}\tilde{H}_k,c_k(\vec{\alpha}_k, t))}
     + \sum_{k=0}^{C-1} \sum_{l=0}^{M-1} \underbrace{(\mathbf{1}_{0,k+1} \otimes B_k) \partial_{{\alpha}_{k,l}} c_k(\vec{\alpha}_k, t)}_{\mathrm{QobjEvo(}\tilde{H}_k,\partial_{{\alpha}_{k,l}} c_k(\vec{\alpha}_k, t))}
\end{gather*}

We have now $1 + C + C \cdot M = O(C \cdot M)$ summands, i.e. the term grows linear with the number of parameters $N_{\mathrm{para}} = C \cdot M$. The size of the matrix however grows with $O(N_{\mathrm{para}} \cdot \mathrm{dim}(A))$.

---
# Example: Hadamard Gate (two level system)
---

\begin{gather*}
    X(t=0) = \mathbf{I} \longrightarrow X(T) = \mathrm{H} = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}
\end{gather*}

We want to get the optimal pulse for implementing the Hadamard gate in a lossy TLS.

In [ ]:
initial = qt.qeye(2)
target = qt.gates.hadamard_transform()

initial = qt.sprepost(initial, initial.dag())
target = qt.sprepost(target, target.dag())

\begin{gather}
    \partial_{t} X(t) =-\frac{i}{\hbar}[H(\vec{\alpha}, t), X(t)]+ \frac{1}{2} \left[2 C X(t) C^\dagger - X(t) C^\dagger C - C^\dagger C X(t)\right] \\
\end{gather}

\begin{gather*}
    H(\vec{\alpha}, t) = \underbrace{
            \frac{1}{2} \left(\omega \sigma_z +  \Delta \sigma_x\right)
        }_{H_d} 
    + H_c(\vec{\alpha}, t) \text{ and }C = \sqrt \gamma a
\end{gather*}

Time independent drift Hamiltonian

In [ ]:
σx = qt.sigmax()
σy = qt.sigmay()
σz = qt.sigmaz()

# energy splitting, tunneling, amplitude damping
ω, Δ, γ = 0.1, 1.0, 0.1

Hd = 1 / 2 * (ω * σz + Δ * σx)

H_d = qt.liouvillian(H=Hd, c_ops=[np.sqrt(γ) * qt.sigmam()])

In [ ]:
π = np.pi
num_ts = 100
interval = TimeInterval(evo_time=2 * π, n_tslots=num_ts)

In [ ]:
Hc = [σx, σy, σz]
H_c = [qt.liouvillian(H) for H in Hc]

init_x = np.ones(num_ts)
init_y = np.ones(num_ts)
init_z = np.ones(num_ts)

H = [H_d, [H_c[0], init_x], [H_c[1], init_y], [H_c[2], init_z]]

Without pulse optimization the resulting gate looks like this.

In [ ]:
init_evo = qt.mesolve(H, initial, interval.tslots)

qt.hinton(init_evo.final_state)

# QuTiP GRAPE

The QOC package consists of a global opimization routine.
Within that multiple local minimizations are performed.
Both routines rely heavily on QuTiPs SESolver to perform efficient time evolution.

The overall optimization routine is adressesed through the `qoc.optimize_pulses` function. We only need to specify the objective, its timeinterval, initial / boundary conditions for the optimization variables (PWC in case of GRAPE) and the optimization algorithm to be used. The `pulse_options` dictionary can have arbitrary (but distinct) keywords.

In [ ]:
res_grape = optimize_pulses(
    objectives=[Objective(initial, H, target)],
    pulse_options={
        "ctrl_x": {
            "guess": init_x,
            "bounds": [-1, 1],
        },
        "ctrl_y": {
            "guess": init_y,
            "bounds": [-1, 1],
        },
        "ctrl_z": {
            "guess": init_z,
            "bounds": [-1, 1],
        },
    },
    time_interval=interval,
    algorithm_kwargs={
        "alg": "GRAPE",
        "fid_err_targ": 0.05,
        "max_iter": 1000,
    },
)

In [ ]:
res_grape

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(15, 4))
ax0.set_title("Initial")
ax1.set_title("Final")
ax2.set_title("Target")

qt.hinton(initial, ax=ax0)
res_grape._final_states = None
res_grape._optimized_objectives = None

qt.hinton(res_grape.final_states[0], ax=ax1)
qt.hinton(target, ax=ax2)

# QuTiP CRAB

In [ ]:
res_crab = optimize_pulses(
    objectives=[Objective(initial, H, target)],
    pulse_options={
        "ctrl_x": {
            "guess": np.zeros(num_ts),
            "bounds": [-1, 1],
        },
        "ctrl_y": {
            "guess": init_y,
            "bounds": [-1, 1],
        },
        "ctrl_z": {
            "guess": init_z,
            "bounds": [-1, 1],
        },
    },
    time_interval=interval,
    algorithm_kwargs={
        "alg": "CRAB",  # changed
        "fid_err_targ": 0.01,
        "disp": True,
        "init_pulse_params": {
            "pulse_action": "add",
        },
    },
    optimizer_kwargs={
        "method": "basinhopping",  # changed
        "max_iter": 0,  # changed
        "seed": 1,
    },
)

In [ ]:
res_crab

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(15, 4))
ax0.set_title("Initial")
ax1.set_title("Final")
ax2.set_title("Target")

qt.hinton(initial, ax=ax0)
qt.hinton(res_crab.final_states[0], ax=ax1)
qt.hinton(target, ax=ax2)

In [ ]:
def plot_result(res, title, y_labels=["$\\sigma_x$", "$\\sigma_y$", "$\\sigma_z$"]):
    fig, ax = plt.subplots(
        len(res.optimized_controls), 1, figsize=(15, 12), sharex=True
    )

    ax[0].set_title(title, fontsize=20)

    for i in range(len(res.optimized_controls)):
        ax[i].xaxis.set_label_text("Time")

        ax[i].yaxis.set_label_text("Control " + y_labels[i])

        ax[i].plot(res.time_interval.tslots, res.guess_controls[i], label="Guess")

        ax[i].plot(
            res.time_interval.tslots, res.optimized_controls[i], label="Optimized"
        )

        ax[i].legend()

In [ ]:
plot_result(res_crab, "CRAB")

# QuTiP GOAT

For continuously defined control functions the procedure is similar.

\begin{gather*}
H_c(\vec{\alpha}, t) =
\underbrace{c_0(\vec{\alpha}_0, t) \sigma_x}_{H_0(t, \vec{\alpha})}  
    +\underbrace{c_1(\vec{\alpha}_1, t) \sigma_y}_{H_1(t, \vec{\alpha})} 
    +\underbrace{c_2(\vec{\alpha}_2, t) \sigma_z}_{H_2(t, \vec{\alpha})} \\ \\
    c_k(\vec{\alpha}, t) = \alpha_{k,0} \cdot \sin(\alpha_{k,1} t + \alpha_{k,2})
\end{gather*}

We start by defining the time dependent / prameterized contorl functions and their derivatives. The signature alway follows the order of f(time, parameter_vector). The derivative functions must have an additional index argument, where the last index always referes to the derivative w.r.t. time.

In [ ]:
def sin(t, α):
    return α[0] * np.sin(α[1] * t + α[2])


def grad_sin(t, α, idx):
    if idx == 0:
        return np.sin(α[1] * t + α[2])
    if idx == 1:
        return α[0] * np.cos(α[1] * t + α[2]) * t
    if idx == 2:
        return α[0] * np.cos(α[1] * t + α[2])
    if idx == 3:
        return α[0] * np.cos(α[1] * t + α[2]) * α[1]  # w.r.t. time

To make each function adressable through a parameter dictionary within QuTiP we label each one with different parameter names.

In [ ]:
def sin_x(t, p):
    return sin(t, p)


def sin_y(t, q):
    return sin(t, q)


def sin_z(t, r):
    return sin(t, r)

The full drift and control Hamiltonian can then be expressed as follows.

In [ ]:
H = [
    H_d,
    [H_c[0], sin_x, {"grad": grad_sin}],
    [H_c[1], sin_y, {"grad": grad_sin}],
    [H_c[2], sin_z, {"grad": grad_sin}],
]

Lets take a look at how the time evolution will look with some initial values.

In [ ]:
p_init = [1, 1, 0]  # amplitude, frequency, phase
q_init = [1, 1, 0]  # q[0] * sin(q[1] * t + q[2])
r_init = [1, 1, 0]

init_evo = qt.mesolve(
    H,
    initial,
    interval.tslots,
    options={"normalize_output": False},
    args={"p": p_init, "q": q_init, "r": r_init},
)

qt.hinton(init_evo.final_state)

To adress the global optimizer we can now include some extra keyword arguments. Notice how the boundaries are now set individually for every single parameter.

In [ ]:
res_goat = optimize_pulses(
    objectives=[Objective(initial, H, target)],
    pulse_options={
        "p": {
            "guess": p_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
        "q": {
            "guess": q_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
        "r": {
            "guess": r_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
    },
    time_interval=interval,
    algorithm_kwargs={
        "alg": "GOAT",  # changed
        "fid_err_targ": 0.01,
    },
    optimizer_kwargs={  # changed
        "max_iter": 5,  # no global optimization
        "seed": 1,  # for reproducibility
    },
)

In [ ]:
res_goat

In [ ]:
plot_result(res_goat, "GOAT")

# QuTiP JAX

Using `qutip-jax` we dont have to provide the derivative, but only need to specify our control functions in a jax compatible way. Note: `qutip-jax` requires control/coefficient functinos to be jit-compiled.

In [ ]:
def sin_jax(t, α):
    return α[0] * jnp.sin(α[1] * t + α[2])

In [ ]:
@jax.jit
def sin_x_jax(t, p, **kwargs):
    return sin_jax(t, p)


@jax.jit
def sin_y_jax(t, q, **kwargs):
    return sin_jax(t, q)


@jax.jit
def sin_z_jax(t, r, **kwargs):
    return sin_jax(t, r)

In [ ]:
H_jax = [H_d, [H_c[0], sin_x_jax], [H_c[1], sin_y_jax], [H_c[2], sin_z_jax]]

In [ ]:
res_jopt = optimize_pulses(
    objectives=[Objective(initial, H_jax, target)],
    pulse_options={
        "p": {
            "guess": p_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
        "q": {
            "guess": q_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
        "r": {
            "guess": r_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
    },
    time_interval=interval,
    algorithm_kwargs={
        "alg": "JOPT",  # changed
        "fid_err_targ": 0.01,
    },
    optimizer_kwargs={
        "max_iter": 0,
        "seed": 1,
    },
)

In [ ]:
res_jopt

In [ ]:
plot_result(res_jopt, "JOPT")

# Time Optimization

GOAT and JOAT additionally offer the possibility to find a optimal pulse duration to get closer to the desired target infidelity.

In [ ]:
res_goat = optimize_pulses(
    objectives=[Objective(initial, H, target)],
    pulse_options={
        "p": {
            "guess": p_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
        "q": {
            "guess": q_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
        "r": {
            "guess": r_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
    },
    time_interval=interval,
    time_options={  # changed
        "guess": 1 / 2 * interval.evo_time,
        "bounds": (0, interval.evo_time),
    },
    algorithm_kwargs={
        "alg": "GOAT",
        "fid_err_targ": 0.01,
    },
    optimizer_kwargs={
        "max_iter": 0,
        "seed": 1,
    },
    integrator_kwargs={  # only for comparison between JOPT and GOAT
        "atol": 1e-5,
        "rtol": 1e-5,
        "method": "dop853",
    },
)

In [ ]:
res_jopt = optimize_pulses(
    objectives=[Objective(initial, H_jax, target)],
    pulse_options={
        "p": {
            "guess": p_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
        "q": {
            "guess": q_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
        "r": {
            "guess": r_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
    },
    time_interval=interval,
    time_options={
        "guess": 1 / 2 * interval.evo_time,
        "bounds": (0, interval.evo_time),
    },
    algorithm_kwargs={
        "alg": "JOPT",
        "fid_err_targ": 0.01,
    },
    optimizer_kwargs={
        "max_iter": 0,
        "seed": 1,
    },
    integrator_kwargs={  # only for comparison between JOPT and GOAT
        "stepsize_controller": PIDController(
            atol=1e-5,
            rtol=1e-5,
        ),
        "solver": Dopri8(),
    },
)

In [ ]:
res_jopt

# Global Optimization

The results already got closer to the desired target. To reach the goal we now include a global optimization.

In [ ]:
res_goat = optimize_pulses(
    objectives=[Objective(initial, H, target)],
    pulse_options={
        "p": {
            "guess": p_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
        "q": {
            "guess": q_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
        "r": {
            "guess": r_init,
            "bounds": [(-1, 1), (0, 1), (0, 2 * π)],
        },
    },
    time_interval=interval,
    time_options={
        "guess": 1 / 2 * interval.evo_time,
        "bounds": (0, interval.evo_time),
    },
    algorithm_kwargs={
        "alg": "GOAT",
        "fid_err_targ": 0.01,
    },
    optimizer_kwargs={
        "max_iter": 100,  # changed
        "seed": 1,
    },
)

In [ ]:
res_goat

# Scale-up

QuTiP offers the possibility to control multiple systems at once.

In [ ]:
N = 2  # number of qubits

initial = qt.tensor([qt.qeye(2)] * N)
target = qt.tensor([qt.gates.hadamard_transform()] * N)

initial = qt.sprepost(initial, initial.dag())
target = qt.sprepost(target, target.dag())

In [ ]:
σx = qt.sigmax()
σy = qt.sigmay()
σz = qt.sigmaz()

ω, Δ, γ, π = 0.1, 1.0, 0.1, np.pi

Hd = 1 / 2 * (ω * σz + Δ * σx)

H_d = qt.liouvillian(
    H=qt.tensor([Hd] * N), c_ops=[np.sqrt(γ) * qt.tensor([qt.sigmam()] * N)]
)

\begin{align*}
    c(t,\vec \alpha) = \sum_{l=0}^{\mathrm{n_{sup}-1}} a_l \sin ( \omega_l t + \phi_l )
\end{align*}

In [ ]:
n_sup = 3
n_var = 3
n_tot = n_sup * n_var

interval = TimeInterval(evo_time=2 * π, n_tslots=1000)

In [ ]:
from qutip_qoc.pulse import SinPulse

import string
import random

sinus = SinPulse(n_sup, n_var)

# Generate random strings for the parameter names (e.g. 'abc')
# we need 3 * N different parameter names, one for each control
params = ["".join(random.choices(string.ascii_letters, k=3)) for _ in range(3 * N)]

pulses, grads = [], []
for param in params:
    # functions with different parameter names
    pulses.append(eval("lambda t, {0}:    sinus.gen_pulse(t, {0})".format(param)))
    grads.append(eval("lambda t, {0}, i: sinus.gen_grad(t, {0}, i)".format(param)))

In [ ]:
Hc = []
id = [qt.qeye(2) for _ in range(N)]

for i in range(N):  # 3N controls, assuming we can control each qubit individually
    sx, sy, sz = id.copy(), id.copy(), id.copy()
    sx[i], sy[i], sz[i] = σx, σy, σz
    Hc.append(qt.tensor(sx))
    Hc.append(qt.tensor(sy))
    Hc.append(qt.tensor(sz))

H_c = [qt.liouvillian(H) for H in Hc]

H = [H_d] + [[hc, pulse, {"grad": grad}] for hc, pulse, grad in zip(H_c, pulses, grads)]

In [ ]:
p_init = np.ones(n_tot)

In [ ]:
pulse_vmap = jax.vmap(sin_jax, in_axes=(None, 0))


def sin_sum(t, α):
    alpha = jnp.reshape(α, (n_sup, n_var))
    return jnp.sum(pulse_vmap(t, alpha), axis=0)


jax_pulses = []
for param in params:
    # functions with different parameter names
    jax_pulses.append(eval("lambda t, {0}, **kwargs: sin_sum(t, {0})".format(param)))

jit_pulses = [jax.jit(pulse) for pulse in jax_pulses]

In [ ]:
H_jax = [H_d] + [[hc, pulse] for hc, pulse in zip(H_c, jit_pulses)]

In [ ]:
p_options = {}
for p in range(len(H_c)):
    p_options[p] = {
        "guess": 0.1 * np.ones(n_tot),
        "bounds": [(-2 * π, 2 * π) for _ in range(n_tot)],
    }

# JOAT

In [ ]:
res_jopt = optimize_pulses(
    objectives=[Objective(initial, H_jax, target)],
    pulse_options=p_options,
    time_interval=interval,
    time_options={
        "guess": interval.evo_time,
        "bounds": (0, 2 * interval.evo_time),
    },
    algorithm_kwargs={
        "alg": "JOPT",
        "fid_err_targ": 0.01,
    },
    optimizer_kwargs={
        "max_iter": 0,
        "seed": 1,
    },
    integrator_kwargs={
        "stepsize_controller": PIDController(
            atol=1e-8,
            rtol=1e-8,
        ),
        "solver": Dopri5(),
    },
)

In [ ]:
res_jopt

# GOAT

In [ ]:
print("system size:", H[0].shape[0])
print("number parameters:", n_tot)
print("number of controls:", len(H) - 1)
print("matrix size:", (1 + n_tot * (len(H) - 1)) * H[0].shape[0])

In [ ]:
res_goat = optimize_pulses(
    objectives=[Objective(initial, H, target)],
    pulse_options=p_options,
    time_interval=interval,
    time_options={
        "guess": interval.evo_time,
        "bounds": (0, 2 * interval.evo_time),
    },
    algorithm_kwargs={
        "alg": "GOAT",
        "fid_err_targ": 0.01,
    },
    optimizer_kwargs={
        "max_iter": 0,
        "seed": 1,
    },
    integrator_kwargs={
        "atol": 1e-8,
        "rtol": 1e-8,
        "method": "dop853",
    },
)

In [ ]:
res_goat

# Global Optimization

# JOAT

In [ ]:
res_jopt = optimize_pulses(
    objectives=[Objective(initial, H_jax, target)],
    pulse_options=p_options,
    time_interval=interval,
    time_options={
        "guess": interval.evo_time,
        "bounds": (0, 2 * interval.evo_time),
    },
    algorithm_kwargs={
        "alg": "JOPT",
        "fid_err_targ": 0.001,  # changed
    },
    optimizer_kwargs={
        "method": "basinhopping",  # changed
        "max_iter": 10,  # changed
        "seed": 1,
    },
    integrator_kwargs={
        "stepsize_controller": PIDController(
            atol=1e-8,
            rtol=1e-8,
        ),
        "solver": Dopri8(),
    },
)

In [ ]:
res_jopt

In [ ]:
plot_result(
    res_jopt, "JOPT", y_labels=["$\\sigma_x$", "$\\sigma_y$", "$\\sigma_z$"] * N
)

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(15, 4))
ax0.set_title("Initial")
ax1.set_title("Final")
ax2.set_title("Target")

qt.hinton(initial, ax=ax0)
qt.hinton(res_jopt.final_states[0], ax=ax1)
qt.hinton(target, ax=ax2)